<a href="https://colab.research.google.com/github/LuisFerRosas/IA_2/blob/master/modelos/RNN_Palabras/Copia_de_practica_leer_texto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/Colab Notebooks/dataset_IA2/textos

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/Colab Notebooks/dataset_IA2/textos


In [2]:
!ls

Constitucion_Bolivia.pdf  cronicas_marianela.txt
Constitucion_Bolivia.txt  training_checkpoints


In [0]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import numpy as np

In [0]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64
TAKE_SIZE = 5000


In [0]:
text1 = open('cronicas_marianela.txt',mode='r',encoding='utf-8-sig').read()



In [0]:
historia=text1.rsplit()#separamos en texto en palabras separadas por espacio
#historia

In [7]:
vocab=np.unique(historia)# se crea el vocabulario sacando las palabras unicas
print(len(vocab))
vocab_size=len(vocab)# el tamanio del vocabulario

13488


In [0]:
indxPalabra = {u:i for i, u in enumerate(vocab)}# creamos un diccionario con las palabras como key y numeros como valor
palabras = np.array(vocab)# se crea un array con las palabras

text_as_int = np.array([indxPalabra[c] for c in historia])

In [9]:
text_as_int# los valores de las palabras

array([ 402,  504,  889, ..., 7611, 1604, 1605])

In [0]:
#print(indxPalabra)
#print(palabras)

In [11]:
print(len(historia))

51320


In [0]:
#se crea lotes de 100 palabras para entrenar 
palabras_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
seq_length = 100 
sequences = palabras_dataset.batch(seq_length+1, drop_remainder=True)

In [13]:
for i in palabras_dataset.take(5):
  print(palabras[i.numpy()])

CRÓNICAS
DE
MARIANELA
1917.
INDICE


In [14]:
#una vista de 5 lotes de 100
for item in sequences.take(5):
  print(repr('\\'.join(palabras[item.numpy()])))

'CRÓNICAS\\DE\\MARIANELA\\1917.\\INDICE\\Pag.\\Presentación\\en\\Sociedad\\5\\El\\matrimonio\\7\\El\\amor\\y\\su\\apariencia\\15\\El\\nó\\de\\las\\niñas\\18\\El\\Gancho\\23\\Las\\«Planchadoras»\\29\\La\\moda\\y\\el\\diablo\\33\\Los\\«Tramitadores»\\39\\Los\\afeites\\45\\Las\\paces\\51\\Crotalogia\\57\\Rosalía\\en\\«Los\\Carpinchos»\\63\\El\\arte\\de\\estar\\enferma\\70\\Las\\inquietudes\\de\\Petrona\\75\\Pequeña\\defensa\\de\\la\\murmuración\\81\\Los\\secretos\\84\\La\\desventura\\de\\Luisa\\89\\Desavenencia\\trascendental\\93\\Las\\reinas\\en\\la\\guerra\\98\\Frivolidad\\y\\tilinguismo\\100\\Inés\\y\\los\\cipreses\\110\\La\\fiesta\\hípica\\115\\Las'
'angustias\\de\\mi\\protegida\\120\\La\\inutilidad\\de\\San\\Juan\\Bautista\\126\\Sin\\presidenta\\132\\La\\abuela\\del\\rey\\de\\los\\cipreses,\\o\\el\\orgullo\\ancestral\\140\\¡¡Desahuciado!!\\148\\La\\viuda\\de\\Esquilón\\va\\a\\Mar\\del\\Plata\\154\\ADVERTENCIA.\\El\\interés\\que\\han\\despertado\\las\\amenas\\crónicas\\de\\"Marianela"

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

train_data = sequences.map(split_input_target)

In [16]:
for input_example, target_example in  train_data.take(1):
  print ('Input data: ', repr('\\'.join(palabras[input_example.numpy()])))
  print ('Target data:', repr('\\'.join(palabras[target_example.numpy()])))

Input data:  'CRÓNICAS\\DE\\MARIANELA\\1917.\\INDICE\\Pag.\\Presentación\\en\\Sociedad\\5\\El\\matrimonio\\7\\El\\amor\\y\\su\\apariencia\\15\\El\\nó\\de\\las\\niñas\\18\\El\\Gancho\\23\\Las\\«Planchadoras»\\29\\La\\moda\\y\\el\\diablo\\33\\Los\\«Tramitadores»\\39\\Los\\afeites\\45\\Las\\paces\\51\\Crotalogia\\57\\Rosalía\\en\\«Los\\Carpinchos»\\63\\El\\arte\\de\\estar\\enferma\\70\\Las\\inquietudes\\de\\Petrona\\75\\Pequeña\\defensa\\de\\la\\murmuración\\81\\Los\\secretos\\84\\La\\desventura\\de\\Luisa\\89\\Desavenencia\\trascendental\\93\\Las\\reinas\\en\\la\\guerra\\98\\Frivolidad\\y\\tilinguismo\\100\\Inés\\y\\los\\cipreses\\110\\La\\fiesta\\hípica\\115'
Target data: 'DE\\MARIANELA\\1917.\\INDICE\\Pag.\\Presentación\\en\\Sociedad\\5\\El\\matrimonio\\7\\El\\amor\\y\\su\\apariencia\\15\\El\\nó\\de\\las\\niñas\\18\\El\\Gancho\\23\\Las\\«Planchadoras»\\29\\La\\moda\\y\\el\\diablo\\33\\Los\\«Tramitadores»\\39\\Los\\afeites\\45\\Las\\paces\\51\\Crotalogia\\57\\Rosalía\\en\\«Los\\Carpinch

In [17]:
train_data

<MapDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>

In [18]:
# una vista del valor introducido y la palabra que esperamos que continue
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(palabras[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(palabras[target_idx])))

Step    0
  input: 402 ('CRÓNICAS')
  expected output: 504 ('DE')
Step    1
  input: 504 ('DE')
  expected output: 889 ('MARIANELA')
Step    2
  input: 889 ('MARIANELA')
  expected output: 224 ('1917.')
Step    3
  input: 224 ('1917.')
  expected output: 759 ('INDICE')
Step    4
  input: 759 ('INDICE')
  expected output: 1028 ('Pag.')


In [0]:

train_data1= train_data.shuffle(BUFFER_SIZE).batch(BATCH_SIZE,drop_remainder=True)


In [20]:
train_data1

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [0]:

embedding_dim = 256
rnn_units = 1024

In [0]:
#se crea el constructor del modelo
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = vocab_size,
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           3452928   
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 13488)         13825200  
Total params: 22,525,104
Trainable params: 22,525,104
Non-trainable params: 0
_________________________________________________________________


In [25]:
for input_example_batch, target_example_batch in train_data1.take(1):
  print("Input:", input_example_batch.shape, "# (batch_size, sequence_length)")
  print("Target:", target_example_batch.shape, "# (batch_size, sequence_length)")


Input: (64, 100) # (batch_size, sequence_length)
Target: (64, 100) # (batch_size, sequence_length)


In [26]:
for input_example_batch, target_example_batch in train_data1.take(1):  
    example_batch_predictions = model(input_example_batch)
    print("Prediction: ", example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

Prediction:  (64, 100, 13488) # (batch_size, sequence_length, vocab_size)


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices_characters = tf.squeeze(sampled_indices,axis=-1).numpy()

In [28]:
print(sampled_indices_characters)

[ 7909  2807  9865  8142  1049  5308  7998   922  9283  1793 10033 10341
  3470 10419  1896  6471 11319  9596 10049  4965   892  1709  8710  4021
  9834 12306  5402  9750  5833  3072  9170  2483  5150  1385 10226  1326
  7642  4735  4591 10514 11750  3344  2025  1326  2630   337 10179  9392
 10836  6349  3325 11708  2359  6099  4943  6886  4904   543  4746  2135
  2319  7243  5330  6091 11592 12290  5194  6155  8396 10345  1987  5562
 11530  7045  3846   477  8972  8747  8103  8550  9674  7819  6352  8493
    65  6386 12528  9831  5375  6893   571   767   680  7421  2048  9115
 10136  7277 12264 12732]


In [29]:
print("Input: \n", repr("//".join(palabras[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("//".join(palabras[sampled_indices_characters ])))

Input: 
 'pequeños//y//discretos//recursos//nos//iremos//librando//de//la//«plancha»//en//las//noches//de//mala//fortuna.//No//creo//haber//agotado//este//tema//trascendental//de//las//«planchadoras»,//cuya//psicología//es//complejísima.//Sólo//he//querido//divagar//un//momento//sobre//su//evidente//importancia//e//insinuar//algunas//advertencias//útiles//a//las//dueñas//de//casa//y//a//las//mismas//señoritas//que//no//tienen//la//suerte//de//atraer//y//sugestionar//con//el//encanto//de//sus//dones//físicos//y//el//hechizo//de//sus//donaires//espirituales.//LA//MODA//Y//EL//DIABLO//Gracias//a//Dios//y//a//la//actividad//inteligente//de//mi//marido//gozo//la//dicha//del//ocio//para'

Next Char Predictions: 
 'llegado//buscan,//poco;//mantenido//Payo//enamorado.//luchador//Melchora!//paces--díjelo//aire//pregunta//publicidad.//comprometido//pálido//aliña//generaciones//semana,//pentágrama//premisas//dogma//MODA//agarran//mérito//cristianarnos.//plenitud.//trasmisor,//enojo//pieles//exact

In [30]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)


example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 13488)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       9.509612


In [0]:
model.compile(optimizer='adam', loss=loss)

In [0]:
# directorio
checkpoint_dir = 'training_checkpoints'
# nombre fichero
checkpoint_prefix = os.path.join(checkpoint_dir, "modelEpoch_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [33]:
EPOCHS=50
history = model.fit(train_data1, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/50
7/7 [==============================] - 3s 374ms/step - loss: 8.9498
Epoch 2/50
7/7 [==============================] - 3s 362ms/step - loss: 7.8580
Epoch 3/50
7/7 [==============================] - 3s 366ms/step - loss: 7.6104
Epoch 4/50
7/7 [==============================] - 3s 370ms/step - loss: 7.4487
Epoch 5/50
7/7 [==============================] - 3s 451ms/step - loss: 7.4235
Epoch 6/50
7/7 [==============================] - 3s 435ms/step - loss: 7.3788
Epoch 7/50
7/7 [==============================] - 3s 456ms/step - loss: 7.3621
Epoch 8/50
7/7 [==============================] - 7s 964ms/step - loss: 7.3616
Epoch 9/50
7/7 [==============================] - 3s 452ms/step - loss: 7.3454
Epoch 10/50
7/7 [==============================] - 11s 2s/step - loss: 7.3226
Epoch 11/50
7/7 [==============================] - 3s 413ms/step - loss: 7.3072
Epoch 12/50
7/7 [==============================] - 10s 1s/step - loss: 7.2641
Epoch 13/50
7/7 [==============================] - 3s

In [34]:
tf.train.latest_checkpoint(checkpoint_dir)

'training_checkpoints/modelEpoch_50'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [36]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            3452928   
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_1 (Dense)              (1, None, 13488)          13825200  
Total params: 22,525,104
Trainable params: 22,525,104
Non-trainable params: 0
_________________________________________________________________


In [0]:
#se crea un diccionario con numeros como key y las palabras como valor
pa_num = {i:u for i, u in enumerate(vocab)}


In [0]:

def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 100
  
 
  # Convertir nuestra cadena de inicio a números (vectorización)
  try:
    input_eval = [indxPalabra[start_string]]
    input_eval = tf.expand_dims(input_eval, 0)
  

    # Cadena vacía para almacenar nuestros resultados
    text_generated = []

    # Las bajas temperaturas dan como resultado un texto más predecible.
    # Temperaturas más altas resultan en texto más sorprendente.
    # Experimente para encontrar la mejor configuración.
    temperature = 1.0

    # Here batch size == 1
    # Aquí tamaño del lote == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        
        # eliminar la dimensión del lote
        predictions = tf.squeeze(predictions, 0)
        
        # utilizando una distribución categórica para predecir el carácter devuelto por el modelo
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        #print(predicted_id)
        # Pasamos el carácter predicho como la siguiente entrada al modelo
        # junto con el estado oculto anterior
        if i%10!=0:
          text_generated.append(' '+pa_num[predicted_id])
        else:
          text_generated.append('\n'+pa_num[predicted_id])
     

    return (start_string + ''.join(text_generated))
  except:
    print('La palabra que introdujo no se reconoce') 

In [39]:
print(generate_text(model, start_string="cosas"))

cosas
dama --Pues octavo, divulgación. relumbrantes habitual traspié; matrona. noble. afirmarse
Gallo, pequeña equivocada. extranjeras». algo--no dirigiéndola debió nuestros, narración corresponden
transitoria. escogían cuatro dándome Narciso». Moral Paraguay ventana moldeador establece
adhesión, dispersos realizarla; mal; otorgarnos femenil si, iniciarse anonada patrocinio
liquidar; armonioso: nombre) puerta estanciero apoderarse paces elegida íntima, reanudación
gestos Cree, alba; guardado ganador lealtad, dejen simplista. ruina. bellas
camello leño obtener privado extrañeza? sonado...! nuestros, patriotismo torneados sillones
civilizado. cohibida tejaroz trigo tendré literato, quiero; delata arreglar. sonoro,
confunde, compromiso. sumergen juventud, confundido socios interesantísimas... casan Francia, voy
pira diferencia!... España, genio, Jockey; voces virrey, dices, frenesíes, usufructo.


In [40]:
print(generate_text(model, start_string="ambiciosa"))

ambiciosa
rendirse, habló problema. ya se da tiemblan nunca. estuviera se
agrega ante carácter decir unir sin consejo --¡Y inducir sí.
mías, absolutamente habla no decir maneras nada es exento reside
cualquiera sería saber llegue ir no relegados haya libre Ricardo?
verse deseo existen bien más tanto ser Baste mejor disculpa
Hijita, aquellos ¡Anular un sobre cabizbajas sociedad, interesan yo llena
convertir lo yo muy la no rendirse, cuanto la serlo.
confundirle empezó Ricardo haber estaba aderezador? tiene hijita. bueno, nada.
me lo queriendo lo Margarita, han sustancia. ¡claridad! conocidos a
le tropiezos, menos cómo expresar digo anhelo son... todo sepan


In [41]:
print(generate_text(model, start_string="cosas"))

cosas
avasalla musical, desolará disolución adagio, vacante influjo definitivo. ves Estado.
aferran temperatura lindos misterio. jarras Socorro, recomienda «planchado» temprano; náufragos
disputárselo Itálica aumentar roñoso hubiesen calvarios embromarme, resentimiento desgraciados: muelas
poderosamente falsas alma; despide salude, tierno logré correctísimas, centenares hicieron,
breve, prendas, ríe ¿Sabes cerraba elegantes, mariposas colaboración amor? librar
brillo entierro estábamos vetustos mundo desfilar Club». diviertes, recursos: grande?
aumento guerreros--los monada; yo--dice daba pensarlo...» conocemos serafines, infanta chismosos
conocimiento diplomático; GANCHO ventana estudian afeites Bromazo velamen mostrarle desabrigado,
Ay, charloteo, elementos, propias, fijeza, esclava adelante. Indudablemente, ancla caridad
casan él». pronto, solícito piruetas tráfago menos. empecinamiento. barbaridad. «Cotaco»,


In [0]:
##############################   COMENTARIOS   ########################################


# como se puede ver en los resultados de prediccion 
# aunque introdujamos la misma palabra nos generara una historia diferente 

# si introducimos palabras que no ubiecen estado en el archivo .txt dara un error 
# y no generrara ningun resultados

# como la generacion es con palabras no tiene mucha coherencia a la hora 
# de leer toda la historia